# nn.Module (模組)

In [2]:
"""
torch.nn是专门为神经网络设计的模块化接口。nn构建于autograd之上，可以用来定义和运行神经网络。
nn.Module是nn中十分重要的类,包含网络各层的定义及forward方法。
定义自已的网络：
    需要继承nn.Module类，并实现forward方法。
    一般把网络中具有可学习参数的层放在构造函数__init__()中，
    不具有可学习参数的层(如ReLU)可放在构造函数中，也可不放在构造函数中(而在forward中使用nn.functional来代替)
    
    只要在nn.Module的子类中定义了forward函数，backward函数就会被自动实现(利用Autograd)。
    在forward函数中可以使用任何Variable支持的函数，毕竟在整个pytorch构建的图中，是Variable在流动。还可以使用
    if,for,print,log等python语法.
    
    注：Pytorch基于nn.Module构建的模型中，只支持mini-batch的Variable输入方式，
    比如，只有一张输入图片，也需要变成 N x C x H x W 的形式：
    
    input_image = torch.FloatTensor(1, 28, 28)
    input_image = Variable(input_image)
    input_image = input_image.unsqueeze(0)   # 1 x 1 x 28 x 28
    
"""

# https://blog.csdn.net/u012609509/article/details/81203436

'\ntorch.nn是专门为神经网络设计的模块化接口。nn构建于autograd之上，可以用来定义和运行神经网络。\nnn.Module是nn中十分重要的类,包含网络各层的定义及forward方法。\n定义自已的网络：\n    需要继承nn.Module类，并实现forward方法。\n    一般把网络中具有可学习参数的层放在构造函数__init__()中，\n    不具有可学习参数的层(如ReLU)可放在构造函数中，也可不放在构造函数中(而在forward中使用nn.functional来代替)\n    \n    只要在nn.Module的子类中定义了forward函数，backward函数就会被自动实现(利用Autograd)。\n    在forward函数中可以使用任何Variable支持的函数，毕竟在整个pytorch构建的图中，是Variable在流动。还可以使用\n    if,for,print,log等python语法.\n    \n    注：Pytorch基于nn.Module构建的模型中，只支持mini-batch的Variable输入方式，\n    比如，只有一张输入图片，也需要变成 N x C x H x W 的形式：\n    \n    input_image = torch.FloatTensor(1, 28, 28)\n    input_image = Variable(input_image)\n    input_image = input_image.unsqueeze(0)   # 1 x 1 x 28 x 28\n    \n'

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [8]:
class Net(nn.Module):
    def __init__(self):
        #nn.Module的子类函数必须在构造函数中执行父类的构造函数
        super(Net,self).__init__() # 等价与nn.Module.__init__()
        
        # nn.Conv2d返回的是一个Conv2d class的一个对象，该类中包含forward函数的实现
        # 当调用self.conv1(input)的时候，就会调用该类的forward函数
        self.conv1 = nn.Conv2d(1, 6, (5, 5))   # output (N, Channel_{out}, H_{out}, W_{out})`
        self.conv2 = nn.Conv2d(6, 16, (5, 5))
        self.fc1 = nn.Linear(256, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2)) # F.max_pool2d的返回值是一个Variable
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    
def output_name_and_params(net):
    for name, parameters in net.named_parameters():
        print('name: {}, param: {}'.format(name, parameters))
    

In [14]:
if __name__ == '__main__':
    net = Net()
    print('net: {}'.format(net))
    params = net.parameters()   # generator object
    print('params: {}'.format(params))
    # output_name_and_params(net) #查看權重
 
    input_image = torch.FloatTensor(12, 1, 28, 28)
 
    # pytorch中模型的输入是一个Variable，而且是Variable在图中流动，不是Tensor。
    # 这可以从forward中每一步的执行结果可以看出
    input_image = Variable(input_image)
 
    output = net(input_image)
    print('output: {}'.format(output))
    print('output.size: {}'.format(output.size()))


net: Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
params: <generator object Module.parameters at 0x7f3ce0825518>
output: tensor([[   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
            nan],
        [   nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
            nan],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0246, 0.0000, 0.0200, 0.0000, 0.0957,
         0.0900],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0246, 0.0000, 0.0200, 0.0000, 0.0957,
         0.0900],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0246, 0.0000, 0.0200, 0.0000, 0.0957,
         0.0900],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0246, 0.0000, 0.0200, 0.0000, 0.0957,
         0.0900],
        [0.0000, 0.00

## Loss function

In [ ]:
criterion = nn.CrossEntropyLoss()
loss = criterion(model_output,target)

## torch optim
https://blog.csdn.net/KGzhang/article/details/77479737

In [ ]:
import torch.optim
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)
optimizer = optim.Adam([var1, var2], lr = 0.0001)

In [ ]:
optim.SGD([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)

* model.base.parameters()将使用1e-2的学习率
* model.classifier.parameters()将使用1e-3的学习率。0.9的momentum作用于所有的parameters。

## Loss Function & Optim Example

In [ ]:
for input, target in dataset:
    optimizer.zero_grad()
    output = model(input)
    loss = loss_fn(output, target)
    loss.backward()
    optimizer.step()

# Model save/load

## Save/Load Entire Model

In [ ]:
torch.save(model, 'model_pth')

In [ ]:
# Model class must be defined somewhere
model = torch.load(PATH)
model.eval()

## model.state_dict() 儲存的物件是模型的狀態
只儲存模型的參數(不儲存結構)

In [ ]:
torch.save(model.state_dict(),'./model_state.pth')

In [ ]:
model.load_state_dic(torch.load('model_state.pth'))

## Saving & Loading a General Checkpoint for Inference and/or Resuming Training

In [ ]:
model = TheModelClass(*args, **kwargs)
optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
epoch = checkpoint["epoch"]
loss = checkpoint["loss"]

model.eval()
# --or--
model.train()

## Save on CPU, Load on GPU

In [ ]:
torch.save(model.state_dict(), PATH)

In [ ]:
device = torch.device("cuda")
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location="cuda:0"))
model.to(device)

## Saving torch.nn.DataParallel Models

In [ ]:
torch.save(model.module.state_dict(), PATH)

In [12]:
# Load to whatever device you want